# Blocking API

Blocking is a technique that makes record linkage scalable. It is achieved by partitioning datasets into groups, called blocks and only comparing records in corresponding blocks. This can reduce the number of comparisons that need to be conducted to find which pairs of records should be linked.

Different blocking techniques have different methods to partition datasets in order to reduce as much number of comparisons as possible while maintain high pair completeness.

In this tutorial, we demonstrate how to use blocking in privacy preserving record linkage. 

Load example Nothern Caroline vote dataset:

In [1]:
import pandas as pd

df_alice = pd.read_csv('data/alice.csv')
df_alice.head()

,recid,givenname,surname,suburb,pc
0,761859,kate,chapman,brighton,4017
1,1384455,lian,hurse,carisbrook,3464
2,1933333,matthew,russo,bardon,4065
3,1564695,lorraine,zammit,minchinbury,2770
4,5971993,ingo,richardson,woolsthorpe,3276


In this dataset, `recid` contains record identifier which can be used to assess the quality of blocking. 

Next step is to config a blocking job. Before we do that, let's look at the blocking methods we are currently supporting:

1. Probabilistic signature (p-sig)
2. LSH based $\Lambda$-fold redundant (lambda-fold)

Let's firstly look at P-sig

### Blocking Methods - Probabilistic signature (p-sig)

This blocking method uses signatures as the blocking key and place only records having same signatures into the same block. 

Let's see an example of configuration for `p-sig`

In [2]:
blocking_config = {
    "type": "p-sig",
    "version": 1,
    "config": {
        "blocking_features": [1],
#         "record-id-col": 0,
        "filter": {
            "type": "ratio",
            "max": 0.02,
            "min": 0.00,
        },
        "blocking-filter": {
            "type": "bloom filter",
            "number-hash-functions": 4,
            "bf-len": 2048,
        },
        "signatureSpecs": [
            [
                 {"type": "characters-at", "config": {"pos": [0]}, "feature-idx": 1},
                 {"type": "characters-at", "config": {"pos": [0]}, "feature-idx": 2},
            ],
            [
                {"type": "metaphone", "feature-idx": 1},
                {"type": "metaphone", "feature-idx": 2},
            ]
        ]
    }
}

**Step1 - Generate Signature**

For a record `r`, a signature is a sub-record deriving from record `r` with a signature strategy. For example, a signature strategy could be taking the initial of names and concatenating letters together i.e. if we have a record with name `"John White"` the signature for this record is `"JW"`.

We provide the following signature strategies:

* feature-value: the signature is generated by returning the selected feature
* characters-at: the signature is generated by selecting a single character or a sequence of characters from selected feature
* metaphone: the signature is generated by phonetic encoding the selected feature using metaphone

The output of this step is a reversed index where keys are generated signatures / blocking key and the values are list of record IDs. The record IDs could be row index or the actual record identifier if it is available in the dataset.

The configuration of signature strategies is in `signatureSpecs` section. For example, in the above configuration, we are going to generate two signatures for each record. The first signature is a combination of 3 signature strategies

```
     {"type": "characters-at", "config": {"pos": [0]}, "feature-idx": 1},
     {"type": "characters-at", "config": {"pos": [0]}, "feature-idx": 2},
     {"type": "feature-value", "feature_idx": 4}

```
It is a combination of name initials and postcode.

The second signature is generated by a combination of 2 signatuer strategies:
```
    {"type": "metaphone", "feature-idx": 1},
    {"type": "metaphone", "feature-idx": 2},
```
That is phonetic encoding of first name and last name.

**Step2 - Filter Too Frequent Signatures**

A signature is assumed to identify a record as uniquely as possible. Therefore, we need to filter out some too frequent signatures since they can uniquely identify the record. On the otherside, we want to be resilient to frequency attack, so we need to filter out too rare signature that only contains very few records. The configuration of filtering is in the `filter` part. For example, in the above configuration, the filter section is configured as:
```
    "filter": {
        "type": "ratio",
        "max": 0.02,
        "min": 0.001,
    }
```
Then we will filter out all signatures / blocks whose number of records is greater than 2% of number of total records or is less than 0.1% of number of total records. 

Note that we also support absoulte filtering configuration i.e. filter by number of counts. For example:

```
    "filter": {
        "type": "count",
        "max": 100,
        "min": 5,
    }
```

**Step3 - Anonymization**

Given we want to do privacy preserving record linkage, the signatures need to be hashed to avoid leaking of PII information. The most frequent used data structure of such encoding is Bloom Filter. Here we use one Bloom Filter and map all filtered signatures into that Bloom Filter. The configuration of Bloom Filter is in `block-filter` section:

```
    "blocking-filter": {
        "type": "bloom filter",
        "number-hash-functions": 20,
        "bf-len": 2048,
    }

```

After anonymization, the signature becomes the set of indices of bits 1 in the bloom filter and hence can preseve the privacy of data for each data provider.

### Carry out Blocking Job

Okay, once you have a good understanding of the P-Sig blocking, we can carry out our blocking job with `blocklib`. Firstly, we need to process the data a bit since `blocklib` only accept list of tuples as input data.

**Step1 - Generate Candidate Blocks for Party A - Alice**

In [3]:
data_alice = df_alice.to_dict(orient='split')['data']
print("Example PII", data_alice[0])

Example PII [761859, 'kate', 'chapman', 'brighton', 4017]


In [4]:
from blocklib import generate_candidate_blocks

block_obj_alice = generate_candidate_blocks(data_alice, blocking_config)

Number of Blocks:   5011
Minimum Block Size: 1
Maximum Block Size: 61
Average Block Size: 1
Median Block Size:  1
Standard Deviation of Block Size:  3.8424497488253815


The statistics of blocks are printed for you to inspect the block distribution and decide if this is a good blocking result. Here both average and median block sizes are 1 which is resilient to frequency attack. 

You can get the blocking instance and blocks/reversed indice in the `block_obj_alice`. Let's get the first signature in the reversed indcies:

In [5]:
print(block_obj_alice.state)
list(block_obj_alice.blocks.keys())[0]

(416, 1650, 836, 1230)

As you can see, the example signature for Alice is a set of indices of bits 1 in Bloom Filter. Next we want to do the same thing for another party - Bob.

**Step2 - Generate Candidate Blocks for Party B - Bob**

In [9]:
df_bob = pd.read_csv('data/bob.csv')
data_bob = df_bob.to_dict(orient='split')['data']
block_obj_bob = generate_candidate_blocks(data_bob, blocking_config)
print(block_obj_bob.state)
print(list(block_obj_bob.blocks.keys())[0])
print(list(block_obj_bob.blocks.values())[1])

Number of Blocks:   5004
Minimum Block Size: 1
Maximum Block Size: 59
Average Block Size: 1
Median Block Size:  1
Standard Deviation of Block Size:  3.841316259098635
(96, 1146, 148, 1198)
[0, 446]


### Generate Final Blocks

Now we have candidate blocks from both parties, we can generate final blocks by only including signatures that appear in both parties. Instead of directly comparing signature, the algorithm will firstly map the list of signatures into a bloom filter for both parites, and then take the and operation on two bloom filter to decide the common .

In [16]:
from blocklib import generate_blocks_2party

filtered_blocks_alice, filtered_blocks_bob = generate_blocks_2party([block_obj_alice, block_obj_bob])
print('Alice: {} out of {} blocks are in common'.format(len(filtered_blocks_alice), len(block_obj_alice.blocks)))
print('Bob:   {} out of {} blocks are in common'.format(len(filtered_blocks_bob), len(block_obj_bob.blocks)))


Alice: 5011 out of 5011 blocks are in common
Bob:   5001 out of 5001 blocks are in common


### Assess Blocking

We can assess the blocking result when we have ground truth . There are two main metrics to assess blocking result:

* reduction ratio: the percentage of reduced number of comparisons
* pair completeness: the percentage of true matches after blocking


In [18]:
from blocklib.assess import assess_blocks_2party


subdata1 = [[x[0]] for x in data_alice]
subdata2 = [[x[0]] for x in data_bob]

rr, pc = assess_blocks_2party([filtered_blocks_alice, filtered_blocks_bob],
                              [subdata1, subdata2])

rr = 0.9961
pc = 1.0
